# Spherical Gaussian experiment: generative results

## Setup

In [1]:
%matplotlib inline

import sys
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
import logging
import corner
from mpl_toolkits.mplot3d import Axes3D

sys.path.append("../../")

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.DEBUG,
)
logger = logging.getLogger(__name__)
# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)


## Load results

In [ ]:
algo_filenames = []
algo_additionals = []
algo_labels = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
add_algo("flow", "_small", "Flow")
add_algo("flow", "_small_long", "Flow (long)")
add_algo("flow", "_small_shallow_long", "Flow (shallow, long)")

add_algo("pie", "_small", "PIE") 
add_algo("pie", "_small_long", "PIE (long)") 
add_algo("pie", "_small_shallow_long", "PIE (shallow, long)") 

add_algo("mf", "_small", "MAD AF")
add_algo("mf", "_small_noprepost", "MAD AF (no pre/post)")
add_algo("mf", "_small_complex", "MAD AF (complex)") 
add_algo("mf", "_small_long", "MAD AF (long)")
add_algo("mf", "_small_shallow_long", "MAD AF (shallow, long)")

add_algo("gamf", "_small_largebs", "OT MAD AF (Sinkhorn)")
add_algo("gamf", "_small_ged_largebs", "OT MAD AF (GED)") 
add_algo("gamf", "_small_ged_largebs_long", "OT MAD AF (GED, long)") 
add_algo("gamf", "_small_ged_largebs_shallow_long", "OT MAD AF (GED, shallow, long)") 

add_algo("pie_specified", "_small", "Prescr. PIE") 
add_algo("pie_specified", "_small_long", "Prescr. PIE (long)") 
add_algo("pie_specified", "_small_shallow_long", "Prescr. PIE (shallow, long)") 

add_algo("mf_specified", "_small", "Prescr. MAD AF") 
add_algo("mf_specified", "_small_long", "Prescr. MAD AF (long)") 
add_algo("mf_specified", "_small_shallow_long", "Prescr. MAD AF (shallow, long)") 

add_algo("gamf_specified", "_small_largebs", "Prescr. OT MAD AF")
add_algo("gamf_specified", "_small_long", "Prescr. OT MAD AF (long)") 
add_algo("gamf_specified", "_small_shallow_long", "Prescr. OT MAD AF (shallow, long)") 


In [45]:
def load(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        results = []
        
        for setup_filename in setup_filenames:
            try:
                results.append(np.load(
                    "{}/{}_2_spherical_gaussian_{}{}_{}.npy".format(
                        result_dir, algo_filename, setup_filename, algo_add, name
                    )
                ))
            except FileNotFoundError as e:
                # print(e)
                
                if shape is None:
                    results.append(None)
                else:
                    results.append(np.nan*np.ones(shape))
            
        all_results.append(results)
    
    return np.asarray(all_results) if numpyfy else all_results

x_gen = load("samples", None, numpyfy=False)
logp_gen = load("samples_likelihood", (10000,))
distance_gen = load("samples_manifold_distance", (10000,))

true_test_log_likelihood = load("true_log_likelihood_test", (1000,))
model_test_log_likelihood = load("model_log_likelihood_test", (1000,))
model_test_reco_error = load("model_reco_error_test", (1000,))

model_ood_log_likelihood = load("model_log_likelihood_ood", (1000,))
model_ood_reco_error = load("model_reco_error_ood", (1000,))


[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.001_reg0001_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.100_reg0001_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.001_reg001_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.100_reg001_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.001_reg01_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.100_reg01_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.001_reg1_samples.npy'
[Errno 2] No such file or directory: '../data/results/flow_2_spherical_gaussian_2_3_0.100_reg1_samples.npy'
[Errno 2] No such file or directory: '../data/results/pie_2_spherical_gaussian_2_3_0.001_reg0001_samples.npy'
[Errno 2] No s

In [46]:
x_test = np.asarray([
    np.load("../data/samples/spherical_gaussian/spherical_gaussian_{}_x_test.npy".format(setup_filename))
    for setup_filename in setup_filenames
])

true_distances = np.abs(np.sum(x_test**2, axis=-1)**0.5 - 1.)

## Calculate metrics

In [48]:
min_logp = -100.
max_distance = 10.

mean_logp_gen = np.mean(np.clip(logp_gen, min_logp, None), axis=2)
mean_distance_gen = np.mean(np.clip(distance_gen, None, max_distance), axis=2)

mean_logp_truth = np.mean(np.clip(true_test_log_likelihood[0], min_logp, None), axis=1)
mean_distance_truth = np.mean(np.clip(true_distances, None, max_distance), axis=1)

In [ ]:
def calculate_roc_auc(x0, x1):
    return np.asarray(
        [
            [
                np.nan if not np.all(np.isfinite(np.hstack((this_x0, this_x1)))) else (
                    np.maximum(
                        roc_auc_score(
                            np.hstack((np.zeros(this_x0.shape[0], dtype=np.int),
                                       np.ones(this_x1.shape[0], dtype=np.int))),
                            np.hstack((this_x0, this_x1)),
                        ),
                        roc_auc_score(
                            np.hstack((np.zeros(this_x0.shape[0], dtype=np.int),
                                       np.ones(this_x1.shape[0], dtype=np.int))),
                            - np.hstack((this_x0, this_x1)),
                        )
                    )
                )
                for this_x0, this_x1 in zip(x0_, x1_)
            ]
            for x0_, x1_ in zip(x0, x1)
        ]
    )


auc_logp = calculate_roc_auc(model_test_log_likelihood, model_ood_log_likelihood)
auc_err = calculate_roc_auc(model_test_reco_error, model_ood_reco_error)
auc_use_err = (auc_err > auc_logp)
auc_best = np.maximum(auc_err, auc_logp)

## Print metrics

In [52]:
def print_results(setup):
    print("{:<15.15s} | {:>8.8s} {:>8.8s} {:>8.8s}".format(algo_label[setup], "Log p", "Distance", "AUC"))
    print("-"*44)
    for label, logp, dist, auc in zip(
        ["Simulator"] + algo_labels,
        [mean_logp_truth] + list(mean_logp_gen),
        [mean_distance_truth] + list(mean_distance_gen),
        [0.0] + list(auc_best)
    ):
        if np.isfinite(logp[setup]) or np.isfinite(dist[setup]):
            print("{:<15.15s} | {:>8.2f} {:>8.3f} {:>8.3f}".format(label, logp[setup], dist[setup], auc[setup]))
        else:
            print("{:<15.15s} |                           ".format(label))

In [53]:
print_results(0)

Algo            |    Log p Distance
-----------------------------------
Truth           |     2.32    0.008
Flow            |     1.72    0.024
Flow (reg 0.001 |    -2.79    0.065
Flow (reg 0.01) |   -30.63    0.225
Flow (reg 0.1)  |   -74.83    0.440
Flow (reg 1)    |   -92.54    0.698
PIE             |     2.54    0.018
PIE (reg 0.001) |    -1.21    0.045
PIE (reg 0.01)  |   -21.59    0.129
PIE (reg 0.1)   |   -88.68    0.529
PIE (reg 1)     |   -88.74    0.544
MF-G            |    -3.54    0.041
MF-G (reg 0.001 |   -11.51    0.054
MF-G (reg 0.01) |   -65.77    0.165
MF-G (reg 0.1)  |   -79.17    0.373
MF-G (reg 1)    |   -88.76    0.536
MF-S            |     2.94    0.000
MF-GS           |     2.87    0.000


In [54]:
print_results(1)

Algo            |    Log p Distance
-----------------------------------
Truth           |     4.68    0.001
Flow            |     3.23    0.023
Flow (reg 0.001 |                  
Flow (reg 0.01) |                  
Flow (reg 0.1)  |                  
Flow (reg 1)    |                  
PIE             |     3.73    0.018
PIE (reg 0.001) |                  
PIE (reg 0.01)  |                  
PIE (reg 0.1)   |                  
PIE (reg 1)     |                  
MF-G            |   -79.64    0.037
MF-G (reg 0.001 |                  
MF-G (reg 0.01) |                  
MF-G (reg 0.1)  |                  
MF-G (reg 1)    |                  
MF-S            |     5.24    0.000
MF-GS           |     5.10    0.000


In [55]:
print_results(2)

Algo            |    Log p Distance
-----------------------------------
Truth           |     0.05    0.081
Flow            |    -0.75    0.098
Flow (reg 0.001 |                  
Flow (reg 0.01) |                  
Flow (reg 0.1)  |                  
Flow (reg 1)    |                  
PIE             |     0.10    0.089
PIE (reg 0.001) |                  
PIE (reg 0.01)  |                  
PIE (reg 0.1)   |                  
PIE (reg 1)     |                  
MF-G            |     0.13    0.038
MF-G (reg 0.001 |                  
MF-G (reg 0.01) |                  
MF-G (reg 0.1)  |                  
MF-G (reg 1)    |                  
MF-S            |     0.64    0.000
MF-GS           |     0.57    0.000
